In [2]:
import pandas as pd
import numpy as np
import setuptools
import openml
from sklearn.linear_model import LinearRegression 
import lightgbm as lgbm
import lightgbmlss
import optuna
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestRegressor
from sklearn.gaussian_process.kernels import Matern
from engression import engression, engression_bagged
import torch
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import mahalanobis
from scipy.stats import norm
from sklearn.metrics import mean_squared_error
from rtdl_revisiting_models import MLP, ResNet, FTTransformer
from properscoring import crps_gaussian, crps_ensemble
import random
import gpytorch
import tqdm.notebook as tqdm
from lightgbmlss.model import *
from lightgbmlss.distributions.Gaussian import *
from drf import drf

In [3]:
# pip install pandas
# pip install setuptools
# pip install openml
# pip install lightgbm
# pip install optuna
# pip install engression


# numpy-1.26.3 pandas-2.1.4 setuptools-69.0.3 openml lightgbm-4.2.0  optuna-3.5.0

In [4]:
#openml.config.apikey = 'FILL_IN_OPENML_API_KEY'  # set the OpenML Api Key
SUITE_ID = 336 # Regression on numerical features
#SUITE_ID = 337 # Classification on numerical features
#SUITE_ID = 335 # Regression on numerical and categorical features
#SUITE_ID = 334 # Classification on numerical and categorical features
benchmark_suite = openml.study.get_suite(SUITE_ID)  # obtain the benchmark suite

task = openml.tasks.get_task(361072)  # download the OpenML task
dataset = task.get_dataset()

X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute)

c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\tasks\functions.py:372: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  warnings.warn(
c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\openml\datasets\functions.py:438: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  warnings.warn(


In [5]:
# New new implementation
N_CLUSTERS=20
# calculate the mean and covariance matrix of the dataset
mean = np.mean(X, axis=0)
cov = np.cov(X.T)
scaler = StandardScaler()

# transform data to compute the clusters
X_scaled = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_scaled)
distances=[]
mahalanobis_dist=[]
counts=[]
ideal_len=len(kmeans.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances.append(np.abs(np.sum(kmeans.labels_==i)-ideal_len))
    counts.append(np.sum(kmeans.labels_==i))
    mean_k= np.mean(X.loc[kmeans.labels_==i,:], axis=0)
    mahalanobis_dist.append(mahalanobis(mean_k, mean, np.linalg.inv(cov)))

dist_df=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist, 'count': counts}, index=np.arange(N_CLUSTERS))
dist_df=dist_df.sort_values('mahalanobis_dist', ascending=False)
dist_df['cumulative_count']=dist_df['count'].cumsum()
dist_df['abs_diff']=np.abs(dist_df['cumulative_count']-ideal_len)

final=(np.where(dist_df['abs_diff']==np.min(dist_df['abs_diff']))[0])[0]
labelss=dist_df.index[0:final+1].to_list()
labels=pd.Series(kmeans.labels_).isin(labelss)
labels.index=X.index
close_index=labels.index[np.where(labels==False)[0]]
far_index=labels.index[np.where(labels==True)[0]]

X_train = X.loc[close_index,:]
X_test = X.loc[far_index,:]
y_train = y.loc[close_index]
y_test = y.loc[far_index]

# calculate the mean and covariance matrix of the dataset
mean_ = np.mean(X_train, axis=0)
cov_ = np.cov(X_train.T)
scaler_ = StandardScaler()

# transform data to compute the clusters
X_train_scaled = scaler_.fit_transform(X_train)

kmeans_ = KMeans(n_clusters=N_CLUSTERS, random_state=0, n_init="auto").fit(X_train_scaled)
distances_=[]
counts_=[]
mahalanobis_dist_=[]
ideal_len_=len(kmeans_.labels_)/5
for i in np.arange(N_CLUSTERS):
    distances_.append(np.abs(np.sum(kmeans_.labels_==i)-ideal_len_))
    counts_.append(np.sum(kmeans_.labels_==i))
    mean_k_= np.mean(X_train.loc[kmeans_.labels_==i,:], axis=0)
    mahalanobis_dist_.append(mahalanobis(mean_k_, mean_, np.linalg.inv(cov_)))

dist_df_=pd.DataFrame(data={'mahalanobis_dist': mahalanobis_dist_, 'count': counts_}, index=np.arange(N_CLUSTERS))
dist_df_=dist_df_.sort_values('mahalanobis_dist', ascending=False)
dist_df_['cumulative_count']=dist_df_['count'].cumsum()
dist_df_['abs_diff']=np.abs(dist_df_['cumulative_count']-ideal_len_)

final_=(np.where(dist_df_['abs_diff']==np.min(dist_df_['abs_diff']))[0])[0]
labelss_=dist_df_.index[0:final_+1].to_list()
labels_=pd.Series(kmeans_.labels_).isin(labelss_)
labels_.index=X_train.index
close_index_=labels_.index[np.where(labels_==False)[0]]
far_index_=labels_.index[np.where(labels_==True)[0]]

X_train_ = X_train.loc[close_index_,:]
X_val = X_train.loc[far_index_,:]
y_train_ = y_train.loc[close_index_]
y_val = y_train.loc[far_index_]

In [21]:
# Convert data to PyTorch tensors
X_train__tensor = torch.tensor(X_train_.values, dtype=torch.float32)
y_train__tensor = torch.tensor(y_train_.values, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

# Convert to use GPU if available
if torch.cuda.is_available():
    X_train__tensor = X_train__tensor.cuda()
    y_train__tensor = y_train__tensor.cuda()
    X_train_tensor = X_train_tensor.cuda()
    y_train_tensor = y_train_tensor.cuda()
    X_val_tensor = X_val_tensor.cuda()
    y_val_tensor = y_val_tensor.cuda()
    X_test_tensor = X_test_tensor.cuda()
    y_test_tensor = y_test_tensor.cuda()

# Create flattened versions of the data
y_val_np = y_val.values.flatten()
y_test_np = y_test.values.flatten()

#### Gaussian process

In [6]:
# Set the random seed for reproducibility
seed=10
np.random.seed(seed)
torch.manual_seed(seed)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood, kernel):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define the learning params
training_iterations = 1000

# Define the kernels
kernels = [
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=0.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=X_train_.shape[1])),
    gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=X_train_.shape[1])),
]

best_crps = float('inf')
best_kernel = None

def train(model,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # Zero backprop gradients
        optimizer.zero_grad()
        # Get output from model
        output = model(X_train_tensor)
        # Calc loss and backprop derivatives
        loss = -mll(output, y_train_tensor)
        loss.backward()
        iterator.set_postfix(loss=loss.item())
        optimizer.step()
        torch.cuda.empty_cache()

for kernel in kernels:
    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = ExactGPModel(X_train_tensor, y_train_tensor, likelihood, kernel)

    # Use the adam optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    # Train the model
    train(model,X_train__tensor,y_train__tensor)
    
    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(X_val_tensor)

    # Calculate CRPS
    y_pred_np = y_pred.mean.numpy().flatten()
    y_pred_std_np = y_pred.stddev.numpy().flatten()

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_val_np))]

    # Calculate the mean CRPS
    mean_crps = np.mean(crps_values)

    # Update the best kernel if the current kernel has a lower CRPS
    if mean_crps < best_crps:
        best_crps = mean_crps
        best_kernel = kernel

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

In [9]:
# Set the random seed for reproducibility
seed=10
np.random.seed(seed)
torch.manual_seed(seed)

class ExactGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactGPModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = best_kernel

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Define the learning params
training_iterations = 1000

# Initialize the Gaussian Process model and likelihood
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactGPModel(X_train_tensor, y_train_tensor, likelihood)

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

# Train the model
train(model,X_train_tensor,y_train_tensor)

# Set the model in evaluation mode
model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = model(X_test_tensor)

# Calculate CRPS
y_pred_np = y_pred.mean.numpy().flatten()
y_pred_std_np = y_pred.stddev.numpy().flatten()

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_test_np))]

# Calculate the mean CRPS
CRPS_GP = np.mean(crps_values)

# Update the best kernel if the current kernel has a lower CRPS
print('CRPS_GP: ', CRPS_GP)

Train:   0%|          | 0/1000 [00:00<?, ?it/s]

CRPS_GP:  59.16435169591171


#### Define train function

In [ ]:
def train(model,criterion,loss_Adam,optimizer,training_iterations,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # making a pridiction in forward pass
        y_train_hat = model(X_train_tensor).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_tensor))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()
        iterator.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()

#### MLP

In [ ]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def MLP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout = trial.suggest_float("dropout", 0, 1)

    MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    dropout=dropout,
    )
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    train(MLP_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_MLP = (MLP_model(X_val_tensor).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_MLP)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_MLP[i], sig=std_dev_error) for i in range(len(y_val_hat_MLP))]

    # Calculate the mean CRPS
    mean_crps = np.mean(crps_values)

    return mean_crps

sampler_MLP = optuna.samplers.TPESampler(seed=10)
study_MLP = optuna.create_study(sampler=sampler_MLP, direction='minimize')
study_MLP.optimize(MLP_opt, n_trials=N_TRIALS)

[I 2024-01-26 12:37:03,009] A new study created in memory with name: no-name-3f5a8d7b-c3d4-4c31-9b57-4689a5b830e9


Train:   0%|          | 0/3769 [00:00<?, ?it/s]

[I 2024-01-26 12:37:48,760] Trial 0 finished with value: 63.01536570936997 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout': 0.6336482349262754, 'n_epochs': 3769, 'learning_rate': 0.002215416944953109}. Best is trial 0 with value: 63.01536570936997.


Train:   0%|          | 0/928 [00:00<?, ?it/s]

[I 2024-01-26 12:38:12,561] Trial 1 finished with value: 2458.6672925365024 and parameters: {'n_blocks': 2, 'd_block': 107, 'dropout': 0.7605307121989587, 'n_epochs': 928, 'learning_rate': 0.0001731515998646626}. Best is trial 0 with value: 63.01536570936997.


In [17]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

MLP_model = MLP(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_MLP.best_params['n_blocks'],
    d_block=study_MLP.best_params['d_block'],
    dropout=study_MLP.best_params['dropout'],
    )
n_epochs=study_MLP.best_params['n_epochs']
learning_rate=study_MLP.best_params['learning_rate']
optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train(MLP_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_MLP = (MLP_model(X_test_tensor).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_MLP)

# Create a normal distribution for each prediction
pred_distributions = [norm(loc=y_test_hat_MLP[i], scale=std_dev_error) for i in range(len(y_test_hat_MLP))]

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_MLP[i], sig=std_dev_error) for i in range(len(y_test_hat_MLP))]

# Calculate the mean CRPS
crps_MLP = np.mean(crps_values)

print("CRPS MLP: ", crps_MLP)

Train:   0%|          | 0/3769 [00:00<?, ?it/s]

CRPS MLP:  48.72480124825327


#### ResNet

In [20]:
N_TRIALS=5

d_out = 1  
d_in=X_train_.shape[1]

def ResNet_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block = trial.suggest_int("d_block", 10, 500)
    dropout1 = trial.suggest_float("dropout1", 0, 1)
    dropout2 = trial.suggest_float("dropout2", 0, 1)
    d_hidden_multiplier=trial.suggest_float("d_hidden_multiplier", 0.5, 3)

    ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block,
    d_hidden=None,
    d_hidden_multiplier=d_hidden_multiplier,
    dropout1=dropout1,
    dropout2=dropout2,
    )
    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    train(ResNet_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_ResNet = (ResNet_model(X_val_tensor).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_ResNet)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_ResNet[i], sig=std_dev_error) for i in range(len(y_val_hat_ResNet))]

    # Calculate the mean CRPS
    crps_ResNet = np.mean(crps_values)

    return crps_ResNet

sampler_ResNet = optuna.samplers.TPESampler(seed=10)
study_ResNet = optuna.create_study(sampler=sampler_ResNet, direction='minimize')
study_ResNet.optimize(ResNet_opt, n_trials=N_TRIALS)

[I 2024-01-26 12:43:21,752] A new study created in memory with name: no-name-ad24174e-75d0-400d-bf3e-7f94c7b4c3d8


Train:   0%|          | 0/1201 [00:00<?, ?it/s]

[I 2024-01-26 12:43:50,929] Trial 0 finished with value: 57.87528155439516 and parameters: {'n_blocks': 4, 'd_block': 20, 'dropout1': 0.6336482349262754, 'dropout2': 0.7488038825386119, 'd_hidden_multiplier': 1.7462675307564761, 'n_epochs': 1201, 'learning_rate': 0.000342425210145967}. Best is trial 0 with value: 57.87528155439516.


Train:   0%|          | 0/119 [00:00<?, ?it/s]

[I 2024-01-26 12:44:10,110] Trial 1 finished with value: 47.17984532196414 and parameters: {'n_blocks': 4, 'd_block': 93, 'dropout1': 0.08833981417401027, 'dropout2': 0.6853598183677972, 'd_hidden_multiplier': 2.8834833654873413, 'n_epochs': 119, 'learning_rate': 0.002412079153798176}. Best is trial 1 with value: 47.17984532196414.


...

[I 2024-01-26 13:36:08,404] Trial 2 finished with value: 9.40510755287084 and parameters: {'n_blocks': 5, 'd_block': 310, 'dropout1': 0.7217553174317995, 'dropout2': 0.29187606817063316, 'd_hidden_multiplier': 2.7944353062823586, 'n_epochs': 3602, 'learning_rate': 0.0029128020748551793}. Best is trial 2 with value: 9.40510755287084.


Train:   0%|          | 0/3127 [00:00<?, ?it/s]

[I 2024-01-26 13:39:29,142] Trial 3 finished with value: 10.218962366967212 and parameters: {'n_blocks': 1, 'd_block': 193, 'dropout1': 0.6741336150663453, 'dropout2': 0.4418331744229961, 'd_hidden_multiplier': 1.5850349833332342, 'n_epochs': 3127, 'learning_rate': 0.0024263012654900037}. Best is trial 2 with value: 9.40510755287084.


Train:   0%|          | 0/1664 [00:00<?, ?it/s]

[I 2024-01-26 14:11:01,927] Trial 4 finished with value: 20.28856863112986 and parameters: {'n_blocks': 4, 'd_block': 305, 'dropout1': 0.8052231968327465, 'dropout2': 0.5216471523936341, 'd_hidden_multiplier': 2.7716222020216708, 'n_epochs': 1664, 'learning_rate': 0.00017544745397857933}. Best is trial 2 with value: 9.40510755287084.


In [19]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

ResNet_model = ResNet(
    d_in=d_in,
    d_out=d_out,
    n_blocks=study_ResNet.best_params['n_blocks'],
    d_block=study_ResNet.best_params['d_block'],
    d_hidden=None,
    d_hidden_multiplier=study_ResNet.best_params['d_hidden_multiplier'],
    dropout1=study_ResNet.best_params['dropout1'],
    dropout2=study_ResNet.best_params['dropout2'],
    )
n_epochs=study_ResNet.best_params['n_epochs']
learning_rate=study_ResNet.best_params['learning_rate']
optimizer=torch.optim.Adam(ResNet_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train(ResNet_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_ResNet = (ResNet_model(X_test_tensor).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_ResNet)

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_ResNet[i], sig=std_dev_error) for i in range(len(y_test_hat_ResNet))]

# Calculate the mean CRPS
crps_ResNet = np.mean(crps_values)

print("CRPS ResNet: ", crps_ResNet)

Train:   0%|          | 0/119 [00:00<?, ?it/s]

CRPS ResNet:  39.39445792693757


#### FFTransformer

In [23]:
N_TRIALS=5

def train_trans(model,criterion,loss_Adam,optimizer,training_iterations,X_train_tensor,y_train_tensor):
    iterator = tqdm.tqdm(range(training_iterations), desc="Train")

    for _ in iterator:
        # making a pridiction in forward pass
        y_train_hat = model(X_train_tensor, None).reshape(-1,)
        # calculating the loss between original and predicted data points
        loss = criterion(y_train_hat, torch.Tensor(y_train_tensor))
        # store loss into list
        loss_Adam.append(loss.item())
        # zeroing gradients after each iteration
        optimizer.zero_grad()
        # backward pass for computing the gradients of the loss w.r.t to learnable parameters
        loss.backward()
        # updating the parameters after each iteration
        optimizer.step()
        iterator.set_postfix(loss=loss.item())
        torch.cuda.empty_cache()

d_out = 1  
d_in=X_train_.shape[1]

def FTTrans_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    n_blocks = trial.suggest_int("n_blocks", 1, 5)
    d_block_multiplier = trial.suggest_int("d_block_multiplier", 1, 25)
    attention_n_heads = trial.suggest_int("attention_n_heads", 1, 20)
    attention_dropout = trial.suggest_float("attention_dropout", 0, 1)
    ffn_d_hidden_multiplier=trial.suggest_float("ffn_d_hidden_multiplier", 0.5, 3)
    ffn_dropout = trial.suggest_float("ffn_dropout", 0, 1)
    residual_dropout = trial.suggest_float("residual_dropout", 0, 1)

    FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=n_blocks,
    d_block=d_block_multiplier*attention_n_heads,
    attention_n_heads=attention_n_heads,
    attention_dropout=attention_dropout,
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=ffn_d_hidden_multiplier,
    ffn_dropout=ffn_dropout,
    residual_dropout=residual_dropout,
    )

    n_epochs=trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)
    weight_decay=trial.suggest_float('weight_decay', 1e-8, 1e-3, log=True)
    optimizer=torch.optim.Adam(MLP_model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    criterion = torch.nn.MSELoss()
    loss_Adam=[]

    train_trans(FTTrans_model,criterion,loss_Adam,optimizer,n_epochs,X_train__tensor,y_train__tensor)

    # Point prediction
    y_val_hat_FTTrans = (FTTrans_model(X_val_tensor, None).reshape(-1,)).detach().numpy()

    # Estimate standard deviation of the prediction error
    std_dev_error = np.std(y_val - y_val_hat_FTTrans)

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_FTTrans[i], sig=std_dev_error) for i in range(len(y_val_hat_FTTrans))]

    # Calculate the mean CRPS
    crps_FTTrans= np.mean(crps_values)

    return crps_FTTrans

sampler_FTTrans = optuna.samplers.TPESampler(seed=10)
study_FTTrans = optuna.create_study(sampler=sampler_FTTrans, direction='minimize')
study_FTTrans.optimize(FTTrans_opt, n_trials=N_TRIALS)

[I 2024-01-26 14:12:45,034] A new study created in memory with name: no-name-89d98ae0-5fc5-4bd5-a718-264130e5fee5


Train:   0%|          | 0/3827 [00:00<?, ?it/s]

In [ ]:
seed=10
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)

FTTrans_model = FTTransformer(
    n_cont_features=d_in,
    cat_cardinalities=[],
    d_out=d_out,
    n_blocks=study_FTTrans.best_params['n_blocks'],
    d_block=study_FTTrans.best_params['d_block'],
    attention_n_heads=study_FTTrans.best_params['attention_n_heads'],
    attention_dropout=study_FTTrans.best_params['attention_dropout'],
    ffn_d_hidden=None,
    ffn_d_hidden_multiplier=study_FTTrans.best_params['ffn_d_hidden_multiplier'],
    ffn_dropout=study_FTTrans.best_params['ffn_dropout'],
    residual_dropout=study_FTTrans.best_params['residual_dropout'],
    )
n_epochs=study_FTTrans.best_params['n_epochs']
learning_rate=study_FTTrans.best_params['learning_rate']
optimizer=torch.optim.Adam(FTTrans_model.parameters(), lr=learning_rate)
criterion = torch.nn.MSELoss()
loss_Adam=[]

train_trans(FTTrans_model,criterion,loss_Adam,optimizer,n_epochs,X_train_tensor,y_train_tensor)

# Point prediction
y_test_hat_FTTrans = (FTTrans_model(X_test_tensor, None).reshape(-1,)).detach().numpy()

# Estimate standard deviation of the prediction error
std_dev_error = np.std(y_test - y_test_hat_FTTrans)

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_FTTrans[i], sig=std_dev_error) for i in range(len(y_test_hat_FTTrans))]

# Calculate the mean CRPS
crps_FTTrans= np.mean(crps_values)

print("CRPS FTTrans: ", crps_FTTrans)

#### Boosted trees, random forest, engression, linear regression

In [247]:
N_TRIALS=2

'''def boosted(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
              'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)}
    
    boosted_tree_model=lgbm.LGBMRegressor(**params)
    boosted_tree_model.fit(X_train_, y_train_)
    y_val_hat_boost=boosted_tree_model.predict(X_val)
    RMSE_boost=np.sqrt(np.mean((y_val-y_val_hat_boost)**2))

    return RMSE_boost

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

boosted_model=lgbm.LGBMRegressor(**study_boost.best_params)

def rf(trial):

    params = {'n_estimators': trial.suggest_int('n_estimators', 100, 500),
              'max_depth': trial.suggest_int('max_depth', 1, 30),
              'max_features': trial.suggest_int('max_features', 1, 30),
              'min_samples_leaf': trial.suggest_int('min_samples_leaf', 10, 100)}
    
    rf_model=RandomForestRegressor(**params)
    rf_model.fit(X_train_, y_train_)
    y_val_hat_rf=rf_model.predict(X_val)
    # Calculate the standard deviation of the residuals
    std_dev = np.std(y_val - y_val_hat_rf)
    # Calculate the CRPS for each prediction
    y_val_np = y_val.values.flatten()
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_rf[i], sig=std_dev) for i in range(len(y_val_np))]
    CRPS_rf = np.mean(crps_values)

    return CRPS_rf

sampler_rf = optuna.samplers.TPESampler(seed=10)
study_rf = optuna.create_study(sampler=sampler_rf, direction='minimize')
study_rf.optimize(rf, n_trials=N_TRIALS)

rf_model=RandomForestRegressor(**study_rf.best_params)'''

torch.manual_seed(10)

# Create lgb dataset
dtrain = lgb.Dataset((X_train_tensor).clone().detach(), label=y_train_.values)

def boosted(trial):

    params = {
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 1, 30),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100),
    }
    opt_params = params.copy()
    n_rounds = opt_params["n_estimators"]
    del opt_params["n_estimators"]
    opt_params['feature_pre_filter']=False

    # Use LightGBMLossGuideRegressor for distributional prediction
    boosted_tree_model = LightGBMLSS(Gaussian(stabilization="None", response_fn="exp", loss_fn="nll"))
    torch.manual_seed(10)
    boosted_tree_model.train(opt_params, dtrain, num_boost_round=n_rounds)

    # Predict both the mean and standard deviation
    pred_params=boosted_tree_model.predict(X_val, pred_type="parameters")
    y_val_hat_boost=pred_params['loc']
    y_val_hat_std = pred_params['scale']

    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_val_hat_boost[i], sig=y_val_hat_std[i]) for i in range(len(y_val))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_boost = optuna.samplers.TPESampler(seed=10)
study_boost = optuna.create_study(sampler=sampler_boost, direction='minimize')
study_boost.optimize(boosted, n_trials=N_TRIALS)

np.random.seed(10)
N_SAMPLES=100

def rf(trial):
    params = {'num_trees': trial.suggest_int('num_trees', 100, 500),
          'mtry': trial.suggest_int('mtry', 1, 30),
          'min_node_size': trial.suggest_int('min_node_size', 10, 100)}
    
    drf_model = drf(**params)
    drf_model.fit(X_train_, y_train_)
    
    # Generate a sample from the drf model for each data point
    y_val_hat=drf_model.predict(newdata = X_val, functional = "quantile", quantiles=list(np.random.uniform(0,1,N_SAMPLES)))

    # Calculate the CRPS for each prediction
    y_val_np = y_val.values.flatten()
    crps_values = [crps_ensemble(y_val_np[i], y_val_hat.quantile[i].reshape(-1)) for i in range(len(y_val_np))]

    # Return the mean CRPS as the objective to be minimized
    return np.mean(crps_values)

sampler_drf = optuna.samplers.TPESampler(seed=10)
study_drf = optuna.create_study(sampler=sampler_drf, direction='minimize')
study_drf.optimize(rf, n_trials=N_TRIALS)

drf_model=drf(**study_drf.best_params)


N_SAMPLES=100
np.random.seed(10)
torch.seed(10)
random.seed(10)
def engressor_NN(trial):

    params = {'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.01, log=True),
              'num_epoches': trial.suggest_int('num_epoches', 100, 1000),
              'num_layer': trial.suggest_int('num_layer', 2, 5),
              'hidden_dim': trial.suggest_int('hidden_dim', 100, 500),}
    params['noise_dim']=params['hidden_dim']
    
    engressor_model=engression(torch.Tensor(np.array(X_train_)), torch.Tensor(np.array(y_train_).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
    # Generate a sample from the engression model for each data point
    y_val_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_val.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_val))]

    # Calculate the CRPS for each prediction
    y_val_np = y_val.values.flatten()
    crps_values = [crps_ensemble(y_val_np[i], np.array(y_val_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_val_np))]

    return np.mean(crps_values)

sampler_engression = optuna.samplers.TPESampler(seed=10)
study_engression = optuna.create_study(sampler=sampler_engression, direction='minimize')
study_engression.optimize(engressor_NN, n_trials=N_TRIALS)

[I 2024-01-23 16:49:49,933] A new study created in memory with name: no-name-a8fc39a0-7487-4f1f-8e81-948c4fcee9b4
[I 2024-01-23 16:49:50,188] Trial 0 finished with value: 18.851961717752452 and parameters: {'learning_rate': 0.12071779104534666, 'n_estimators': 108, 'reg_lambda': 0.005044685709888605, 'max_depth': 23, 'min_child_samples': 55}. Best is trial 0 with value: 18.851961717752452.
[I 2024-01-23 16:49:50,485] Trial 1 finished with value: 21.13441405830955 and parameters: {'learning_rate': 0.004043145805966843, 'n_estimators': 179, 'reg_lambda': 0.0699481785242808, 'max_depth': 6, 'min_child_samples': 18}. Best is trial 0 with value: 18.851961717752452.
[I 2024-01-23 16:49:51,116] Trial 2 finished with value: 18.918708311325076 and parameters: {'learning_rate': 0.07075637776590661, 'n_estimators': 482, 'reg_lambda': 1.08526150100961e-08, 'max_depth': 16, 'min_child_samples': 83}. Best is trial 0 with value: 18.851961717752452.
[I 2024-01-23 16:49:51,662] Trial 3 finished with va

[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

[I 2024-01-23 16:50:00,542] Trial 0 finished with value: 9.184574701452648 and parameters: {'num_trees': 409, 'mtry': 1, 'min_node_size': 67}. Best is trial 0 with value: 9.184574701452648.


[0.5781364298824675, 0.8539337505004864, 0.06809727353795003, 0.46453080777933253, 0.7819491186191484, 0.7186028103822503, 0.5860219800531759, 0.037094413234407875, 0.350656391283133, 0.563190684492745, 0.29972987242456284, 0.5123341532735493, 0.6734669252847205, 0.1591937333780935, 0.05047767015399762, 0.33781588706467947, 0.10806377277945256, 0.17890280857109042, 0.8858270961677057, 0.3653649712141158, 0.21876934917953672, 0.7524961702186028, 0.10687958439356915, 0.7446032407755606, 0.46978529344049447, 0.5982556712791092, 0.14762019228529766, 0.18403482209315125, 0.6450721264682419, 0.048628006263405577, 0.24861250780276944, 0.5424085162280042, 0.2267733432700092, 0.3814115349046321, 0.9222327869035463, 0.9253568728677768, 0.566749924575, 0.5334708849890026, 0.014860024633228108, 0.977899263402005, 0.5730289040331858, 0.791756996276624, 0.5615573602763689, 0.8773352415649347, 0.5841958285306755, 0.7088498263689552, 0.14853345135645857, 0.4284507389678964, 0.6938900663424117, 0.10461

[I 2024-01-23 16:50:10,538] Trial 1 finished with value: 7.119592117254569 and parameters: {'num_trees': 400, 'mtry': 15, 'min_node_size': 30}. Best is trial 1 with value: 7.119592117254569.


...

[I 2024-01-23 16:54:59,009] Trial 2 finished with value: 25.963044786491658 and parameters: {'learning_rate': 0.07075637776590661, 'num_epoches': 959, 'num_layer': 2, 'hidden_dim': 76, 'noise_dim': 91}. Best is trial 1 with value: 4.780338391133786.


Running on CPU.

Residual blocks (skip-connections) are typically recommended for more than 2 layers; turn it on by setting resblock=True.
Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.5532,  E(|Y-Yhat|): 1.0617,  E(|Yhat-Yhat'|): 1.0170
[Epoch 100 (13%), batch 6] energy-loss: 0.2096,  E(|Y-Yhat|): 0.4413,  E(|Yhat-Yhat'|): 0.4635
[Epoch 200 (27%), batch 6] energy-loss: 0.1857,  E(|Y-Yhat|): 0.4046,  E(|Yhat-Yhat'|): 0.4380
[Epoch 300 (40%), batch 6] energy-loss: 0.1835,  E(|Y-Yhat|): 0.3933,  E(|Yhat-Yhat'|): 0.4196
[Epoch 400 (53%), batch 6] energy-loss: 0.2127,  E(|Y-Yhat|): 0.4000,  E(|Yhat-Yhat'|): 0.3746
[Epoch 500 (67%), batch 6] energy-loss: 0.2109,  E(|Y-Yhat|): 0.3921,  E(|Yhat-Yhat'|): 0.3626
[Epoch 600 (80%

[I 2024-01-23 16:57:24,963] Trial 3 finished with value: 11.685079293248108 and parameters: {'learning_rate': 0.044997613517186334, 'num_epoches': 750, 'num_layer': 3, 'hidden_dim': 96, 'noise_dim': 86}. Best is trial 1 with value: 4.780338391133786.


Running on CPU.

Residual blocks (skip-connections) are typically recommended for more than 2 layers; turn it on by setting resblock=True.
Data is standardized for training only; the printed training losses are on the standardized scale. 
However during evaluation, the predictions, evaluation metrics, and plots will be on the original scale.

Training based on mini-batch gradient descent with a batch size of 1000.
[Epoch 1 (0%), batch 6] energy-loss: 0.2918,  E(|Y-Yhat|): 0.5928,  E(|Yhat-Yhat'|): 0.6020
[Epoch 100 (43%), batch 6] energy-loss: 0.2099,  E(|Y-Yhat|): 0.4512,  E(|Yhat-Yhat'|): 0.4825
[Epoch 200 (87%), batch 6] energy-loss: 0.1959,  E(|Y-Yhat|): 0.4321,  E(|Yhat-Yhat'|): 0.4724

Training loss on the original (non-standardized) scale:
	Energy-loss: 1.0516,  E(|Y-Yhat|): 2.0883,  E(|Yhat-Yhat'|): 2.0735

Prediction-loss E(|Y-Yhat|) and variance-loss E(|Yhat-Yhat'|) should ideally be equally large
-- consider training for more epochs or adjusting hyperparameters if there is a

[I 2024-01-23 16:58:24,716] Trial 4 finished with value: 3.7364615733368005 and parameters: {'learning_rate': 0.029128020748551788, 'num_epoches': 228, 'num_layer': 3, 'hidden_dim': 84, 'noise_dim': 72}. Best is trial 4 with value: 3.7364615733368005.


In [266]:
N_SAMPLES=100
seed=10
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
random.seed(seed)

'''boosted_model.fit(X_train, y_train)
y_test_hat_boosted=boosted_model.predict(X_test)
RMSE_boosted=np.sqrt(np.mean((y_test-y_test_hat_boosted)**2))

rf_model.fit(X_train, y_train)
y_test_hat_rf=rf_model.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_rf)
# Calculate the CRPS for each prediction
y_test_np = y_test.values.flatten()
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_rf[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_rf = np.mean(crps_values)'''

drf_model.fit(X_train, y_train)
# Generate a sample from the drf model for each data point
y_test_hat_drf=drf_model.predict(newdata = X_test, functional = "quantile", quantiles=list(np.random.uniform(0,1,N_SAMPLES)))
# Calculate the CRPS for each prediction
y_test_np = y_test.values.flatten()
crps_values = [crps_ensemble(y_test_np[i], y_test_hat_drf.quantile[i].reshape(-1)) for i in range(len(y_test_np))]
# Return the mean CRPS as the objective to be minimized
CRPS_rf=np.mean(crps_values)

lin_reg=LinearRegression()
lin_reg.fit(X_train, y_train)
y_test_hat_linreg=lin_reg.predict(X_test)
# Calculate the standard deviation of the residuals
std_dev = np.std(y_test - y_test_hat_linreg)
# Calculate the CRPS for each prediction
y_test_np = y_test.values.flatten()
crps_values = [crps_gaussian(y_test_np[i], mu=y_test_hat_linreg[i], sig=std_dev) for i in range(len(y_test_np))]
CRPS_linreg = np.mean(crps_values)

params=study_engression.best_params
params['noise_dim']=params['hidden_dim']
engressor_model=engression(torch.Tensor(np.array(X_train)), torch.Tensor(np.array(y_train).reshape(-1,1)), lr=params['learning_rate'], num_epoches=params['num_epoches'],num_layer=params['num_layer'], hidden_dim=params['hidden_dim'], noise_dim=params['noise_dim'], batch_size=1000)
# Generate a sample from the engression model for each data point
y_test_hat_engression_samples = [engressor_model.sample(torch.Tensor(np.array([X_test.values[i]])), sample_size=N_SAMPLES) for i in range(len(X_test))]
# Calculate the CRPS for each prediction
y_test_np = y_test.values.flatten()
crps_values = [crps_ensemble(y_test_np[i], np.array(y_test_hat_engression_samples[i]).reshape(-1,)) for i in range(len(y_test_np))]
CRPS_engression=np.mean(crps_values)


print("CRPS linear regression: ",CRPS_linreg)
print("RMSE boosted trees", RMSE_boosted)
print("CRPS random forest", CRPS_rf)
print("CRPS engression", CRPS_engression)

[0.771320643266746, 0.0207519493594015, 0.6336482349262754, 0.7488038825386119, 0.4985070123025904, 0.22479664553084766, 0.19806286475962398, 0.7605307121989587, 0.16911083656253545, 0.08833981417401027, 0.6853598183677972, 0.9533933461949365, 0.003948266327914451, 0.5121922633857766, 0.8126209616521135, 0.6125260668293881, 0.7217553174317995, 0.29187606817063316, 0.9177741225129434, 0.7145757833976906, 0.5425443680112613, 0.14217004760152696, 0.3733407600514692, 0.6741336150663453, 0.4418331744229961, 0.4340139933332937, 0.6177669784693172, 0.5131382425543909, 0.6503971819314672, 0.6010389534045444, 0.8052231968327465, 0.5216471523936341, 0.9086488808086682, 0.3192360889885453, 0.09045934927090737, 0.30070005663620336, 0.11398436186354977, 0.8286813263076767, 0.04689631938924976, 0.6262871483113925, 0.5475861559192435, 0.8192869956700687, 0.1989475396788123, 0.8568503024577332, 0.3516526394320879, 0.7546476915298572, 0.2959617068796787, 0.8839364795611863, 0.3255116378322488, 0.165015

In [ ]:
#### EXTRA

## EXTRA

#### Gaussian process with stochastic variational inference

In [ ]:
N_TRIALS=2

def SVGP_opt(trial):

    seed=10
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)

    class SVGPMODEL(gpytorch.models.ApproximateGP):
        def __init__(self, inducing_points):
            variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
            variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
            super(SVGPMODEL, self).__init__(variational_strategy)
            self.mean_module = gpytorch.means.ConstantMean()
            self.covar_module = gpytorch.kernels.ScaleKernel(
                gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1]),
                ard_num_dims=inducing_points.shape[1]
            )

        def forward(self, x):
            mean_x = self.mean_module(x)
            covar_x = self.covar_module(x)
            return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_.values, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_.values, dtype=torch.float32)

    # Initialize the Gaussian Process model and likelihood
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = SVGPMODEL(X_train_tensor)

    # Set the model in training mode
    model.train()
    likelihood.train()

    # Define the learning params
    n_epochs=5 #trial.suggest_int('n_epochs', 100, 5000)
    learning_rate=trial.suggest_float('learning_rate', 0.0001, 0.05, log=True)

    # Use the negative log likelihood as the loss
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=y_train_tensor.numel())
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Training loop
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        output = model(X_train_tensor)
        loss = -mll(output, y_train_tensor)
        loss.backward()
        optimizer.step()

    # Set the model in evaluation mode
    model.eval()
    likelihood.eval()

    # Make predictions on the validation set
    with torch.no_grad(), gpytorch.settings.fast_pred_var():
        y_pred = model(torch.tensor(X_val.values, dtype=torch.float32))

    # Calculate CRPS
    y_val_np = y_val.values.flatten()
    y_pred_np = y_pred.mean.numpy().flatten()
    y_pred_std_np = y_pred.stddev.numpy().flatten()
    
    # Calculate the CRPS for each prediction
    crps_values = [crps_gaussian(y_val_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_val_np))]

    # Calculate the mean CRPS
    mean_crps= np.mean(crps_values)
    
    return mean_crps

sampler_SVGP = optuna.samplers.TPESampler(seed=10)
study_SVGP = optuna.create_study(sampler=sampler_SVGP, direction='minimize')
study_SVGP.optimize(SVGP_opt, n_trials=N_TRIALS)

[I 2024-01-22 17:10:06,859] A new study created in memory with name: no-name-bea4cf5f-3031-4dec-a045-a078b921cc60


[W 2024-01-22 17:18:36,997] Trial 0 failed with parameters: {'lengthscale': 0.08747537025773001, 'learning_rate': 0.00011376505702653915} because of the following error: AxisError(-1, 0, None).
Traceback (most recent call last):
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_14344\2994460075.py", line 77, in SVGP_opt
    crps_values = [CRPS(y_val_np[i], pred_distributions[i]) for i in range(len(y_val_np))]
  File "C:\Users\dalma\AppData\Local\Temp\ipykernel_14344\2994460075.py", line 77, in <listcomp>
    crps_values = [CRPS(y_val_np[i], pred_distributions[i]) for i in range(len(y_val_np))]
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\CRPS\CRPS.py", line 109, in __init__
    self.fc = np.sort(ensemble_members)
  File "c:\Users\dalma\Desktop\THESIS_ETH_NEW\CODE\.venv\lib\site-packages\numpy\cor

AxisError: axis -1 is out of bounds for array of dimension 0

In [ ]:
# Access the best parameters
best_params_SVGP = study_SVGP.best_params
lengthscale = best_params_SVGP['lengthscale']
n_epochs = best_params_SVGP['n_epochs']
learning_rate = best_params_SVGP['learning_rate']

class SVGPMODEL(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(SVGPMODEL, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=1.5, ard_num_dims=inducing_points.shape[1], lengthscale=lengthscale),
            ard_num_dims=inducing_points.shape[1]
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# Convert data to PyTorch tensors
X_tensor = torch.tensor(X_train, dtype=torch.float32)
y_tensor = torch.tensor(y_train, dtype=torch.float32)

# Initialize the final Gaussian Process model with the best parameters
likelihood = gpytorch.likelihoods.GaussianLikelihood()
final_model = SVGPMODEL(X_tensor)

# Set the model in training mode
final_model.train()
likelihood.train()

# Use the negative log likelihood as the loss
mll = gpytorch.mlls.VariationalELBO(likelihood, final_model, num_data=y_tensor.numel())
optimizer = torch.optim.Adam(final_model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(n_epochs):
    optimizer.zero_grad()
    output = final_model(X_tensor)
    loss = -mll(output, y_tensor)
    loss.backward()
    optimizer.step()

# Set the final model in evaluation mode
final_model.eval()
likelihood.eval()

# Make predictions on the validation set
with torch.no_grad(), gpytorch.settings.fast_pred_var():
    y_pred = final_model(torch.tensor(X_test.values, dtype=torch.float32))


# Calculate CRPS
y_test_np = y_test.values.flatten()
y_pred_np = y_pred.mean.numpy().flatten()
y_pred_std_np = y_pred.stddev.numpy().flatten()

# Calculate the CRPS for each prediction
crps_values = [crps_gaussian(y_test_np[i], mu=y_pred_np[i], sig=y_pred_std_np[i]) for i in range(len(y_test_np))]

# Calculate the mean CRPS
CRPS_SVGP= np.mean(crps_values)

print("CRPS SVGP: ", CRPS_SVGP)

NameError: name 'study_SVGP' is not defined